# Imports

In [ ]:
import copy
import glob
import itertools
import numpy as np
import h5py
import scipy
import sys
import tqdm
import verdict
import os
import unyt

In [ ]:
import matplotlib as mpl
from matplotlib.lines import Line2D
import matplotlib.gridspec as gridspec
import matplotlib.patheffects as patheffects
from mpl_toolkits.axes_grid1 import make_axes_locatable 
import matplotlib.image as mpimg
import palettable

In [ ]:
import kalepy as kale

In [ ]:
import galaxy_dive.analyze_data.halo_data as halo_data
import galaxy_dive.plot_data.plotting as plotting

In [ ]:
from py2tex import py2tex

In [ ]:
import trove
import trove.config_parser

# Load Data

In [ ]:
pm = {
    'sims_to_plot': [ [ 'm12i_md', 'm12b_md', ], [ 'm12c_md', 'm11d_md' ] ],
    'sims_to_plot_flat': ['m12i_md', 'm12b_md', 'm12c_md', 'm11d_md' ],
#     'sims_to_plot': [ [ 'm11d_md', 'm11e_md', ], [ 'm11h_md', 'm11i_md' ], [ 'm11q_md', 'm11a' ], [ 'm11c', 'm12i_md' ] ],
}

In [ ]:
config_fp = '/home1/03057/zhafen/papers/Hot-Accretion-in-FIRE/analysis/hot_accretion.trove'
cp = trove.config_parser.ConfigParser( config_fp )
pm = trove.link_params_to_config(
    config_fp,
    variation = 'm12i_md',
    **pm
)

In [ ]:
sims_to_plot = pm['sims_to_plot']

In [ ]:
if 'sims_to_plot_flat' not in pm:
    sims_to_plot_flat = []
    for row in sims_to_plot[::-1]:
        sims_to_plot_flat += row
else:
    sims_to_plot_flat = pm['sims_to_plot_flat']

In [ ]:
data_filepath = os.path.join( pm['processed_data_dir'], 'summary.hdf5' )
data = verdict.Dict.from_hdf5( data_filepath )

In [ ]:
h_param = .702

In [ ]:
mvir = verdict.Dict({})
mstar = verdict.Dict({})
rgal = verdict.Dict({})
rvir = verdict.Dict({})
for key in tqdm.tqdm( cp.variations ):
    
    pm_i = trove.link_params_to_config(
        '/home1/03057/zhafen/papers/Hot-Accretion-in-FIRE/analysis/hot_accretion.trove',
        variation = key,
    )
    
    h_data = halo_data.HaloData(
        data_dir = pm_i['halo_data_dir'],
        mt_kwargs = { 'tag': 'smooth' },
    )
    
    mvir[key] =  h_data.get_mt_data( 'Mvir', snums=[600] )[0]/h_param
    mstar[key] = h_data.get_mt_data( 'M_star', snums=[600] )[0]/h_param
    
    rstar_z0 = h_data.get_mt_data( 'Rstar0.5', snums=[600] )/h_param
    rgal[key] = 4. * rstar_z0
    rvir[key] = h_data.get_mt_data( 'Rvir', snums=[600] )/h_param

# Edge-on Images

In [ ]:
# Set plot parameters
x_lim_rgal = [ -2, 2 ]
y_lim_rgal = [ -1, 1 ]
panel_width = 8.

In [ ]:
# Derived plot parameters
aspect = ( y_lim_rgal[1] - y_lim_rgal[0] ) / ( x_lim_rgal[1] - x_lim_rgal[0] )
n_rows = len( sims_to_plot )
n_cols = len( sims_to_plot[0] )

# Somehow the aspect isn't quite right. This fixes that.
correction_factor = 1.025

In [ ]:
fig = plt.figure( figsize=(panel_width*n_cols, panel_width*aspect*correction_factor*n_rows), facecolor='w' )
fig.subplots_adjust( hspace=0.0001, wspace=0.0001 )
ax_dict = fig.subplot_mosaic( sims_to_plot )

for k, row in enumerate( sims_to_plot ):
    
    for j, sim in enumerate( row ):

        ax = ax_dict[sim]
        
        # Image dir
        data_dir = os.path.join( pm['root_data_dir'], sim )
        image_dir = os.path.join( data_dir, 'firestudio', 'Plots', 'GasTwoColour' )

        # Image filepath
        image_glob = os.path.join( image_dir, 'edge_on*.png' )
        image_fps = glob.glob( image_glob )
        assert len( image_fps ) == 1
        image_fp = image_fps[0]

        # Metadata filepath
        metadata_glob = os.path.join( image_dir, 'edge_on*hdf5' )
        metadata_fps = glob.glob( metadata_glob )
        assert len( metadata_fps ) == 1
        metadata_fp = metadata_fps[0]

        # Load image and metadata
        img = mpimg.imread( image_fp )
        metadata = verdict.Dict.from_hdf5( metadata_fp )

        # Rotate image and set up xs and ys
        img = np.rot90( img )
        xs = np.linspace( metadata['Ymin'], metadata['Ymax'], img.shape[1] )
        ys = np.linspace( metadata['Xmin'], metadata['Xmax'], img.shape[0] )

        # Limits in physical kpc
        x_lim = rgal[key] * np.array( x_lim_rgal )
        y_lim = rgal[key] * np.array( y_lim_rgal )

        # Select only part of the image
        in_bound_x = ( xs > x_lim[0] )& ( xs < x_lim[1] )
        in_bound_y = ( ys > y_lim[0] )& ( ys < y_lim[1] )
        img = img[:,in_bound_x][in_bound_y,:]
        xs = xs[in_bound_x]
        ys = ys[in_bound_y]

        # Reset limit to avoid weird whitespace
        x_lim = [ xs[0], xs[-1] ]
        y_lim = [ ys[0], ys[-1] ]

        extent = [ xs[0], xs[-1], ys[0], ys[-1] ] 
        
        ax.imshow(
            img,
            extent = extent,
            aspect = 'equal',
        )
        
        
        # Change colors
        ax.set_facecolor( 'k' )
        plt.setp( ax.spines.values(), color='w' )
        [m.set_linewidth(2) for m in ax.spines.values()]
                
        # Limits
        ax.set_xlim( x_lim )
        ax.set_ylim( y_lim )
        
        # Ticks
        ax.tick_params( 
            'both', 
            left = False,
            labelleft = False,
            bottom = False,
            labelbottom = False,
        )
        
        # Scale bar
        def round_down(num, divisor):
            return num - (num%divisor)
        size = int( round_down( rgal[key], 10 ) )
        if np.isclose( size, 0. ):
            size = 10.
        xy = ( -size, 0.9 * y_lim[0] )
        ax.plot(
            xy[0] + np.array([ 0., size ]),
            [ xy[1], xy[1] ],
            linewidth = 5,
            color = 'w',
        )
        ax.annotate(
            text = '{:.2g} kpc'.format( size ),
            xy = xy,
            xycoords = 'data',
            xytext = ( 5, 10 ),
            textcoords = 'offset points',
            va = 'bottom',
            ha = 'left',
            color = 'w',
            fontsize = 24,
        )
        
        # Sim name label
        sim_label = r'$M_{\rm vir}=' + py2tex.to_tex_scientific_notation( mvir[sim], 2 ) +  r'M_\odot$' + '\n'
        sim_label += sim.split( '_' )[0]
        ax.annotate(
            text = sim_label,
            xy = ( 0, 1 ),
            xycoords = 'axes fraction',
            xytext = ( 7, -7 ),
            textcoords = 'offset points',
            ha = 'left',
            va = 'top',
            fontsize = 22,
            color = 'w',
        )
        # Thin disk fraction
        fraction_label = r'$f_{\rm thin\,disk,\,recent}$ = ' + '{:.2g}'.format( np.round( data['thin_disk_fraction_recent'][sim], 2 ) ) # + '\n'
        ax.annotate(
            text = fraction_label,
            xy = ( 1, 1 ),
            xycoords = 'axes fraction',
            xytext = ( -7, -7 ),
            textcoords = 'offset points',
            ha = 'right',
            va = 'top',
            fontsize = 22,
            color = 'w',
        )

        
plotting.save_fig(
    out_dir = pm['figure_dir'],
    save_file = 'stars.pdf',
    fig = fig,
)

# $R_{T = 10^5 {\rm K}}$ Distributions

In [ ]:
qual_cmap = palettable.cartocolors.qualitative.Vivid_10.mpl_colors
cmap = palettable.matplotlib.Viridis_20.mpl_colormap
norm = matplotlib.colors.Normalize( vmin=data['thin_disk_fraction'].keymin()[1], vmax=data['thin_disk_fraction'].keymax()[1] )

## One Panel

In [ ]:
linestyles = {
    'm11': '--',
    'm12': '-',
}

In [ ]:
label_locs = {
    'm12i_md': ( rgal['m12c_md'] / rvir['m12c_md'], 0.01 ),
    'm12b_md': ( rgal['m12c_md'] / rvir['m12c_md'], 0.007 ),
    'm12c_md': ( rgal['m12c_md'] / rvir['m12c_md'], 0.004 ),
#     'm11d_md': ( rgal['m11d_md'] / rvir['m11d_md'], 0.04 ),
}

### Scaled by Rvir

In [ ]:
scale = 1.
fig = plt.figure( figsize=(16,5), facecolor='w' )
ax = plt.gca()

for i, sim in enumerate( sims_to_plot_flat ):
    ax.plot(
        data['R1e5K']['points'][sim] / rvir[sim],
        data['R1e5K']['pdf'][sim],
        linewidth = 5,
        c = qual_cmap[i],
#         c = cmap( norm( data['thin_disk_fraction'][sim] ) ),
        label = sim.split( '_' )[0],
        zorder = np.log10( mvir[sim] ),
        linestyle = linestyles[sim[:3]]
    )
    
    # Lines for the galaxy radius
    ax.axvline(
        rgal[sim] / rvir[sim],
        linewidth = 2,
        c = qual_cmap[i],
#         c = cmap( norm( data['thin_disk_fraction'][sim] ) ),
        linestyle = linestyles[sim[:3]],
        zorder = np.log10( mvir[sim] ),
    )

#     # Annotate the lines
#     if sim in label_locs:
#         ax.annotate(
#             text = r'$R_{\rm gal}$',
#             xy = label_locs[sim],
#             xytext = ( 5, -5 ),
#             xycoords = 'data',
#             textcoords = 'offset points',
#             fontsize = 20,
#             color = qual_cmap[i],
#         )

    
ax.legend(
    prop = { 'size': 18 },
)
    
ax.set_xlim( 0, 0.5 )
# ax.set_ylim( 0, 0.045 )

ax.set_xlabel( r'$R_{T=10^5 {\rm K}}$ / $R_{\rm vir}$', fontsize=22 )
ax.set_ylabel( r'PDF( $R_{T=10^5 {\rm K}}$ )', fontsize=22 )

# plotting.save_fig(
#     out_dir = pm['figure_dir'],
#     save_file = 'R1e5K.pdf',
#     fig = fig,
# )

### Physical

In [ ]:
label_locs = {
    'm12i_md': ( rgal['m12i_md'], 0.016, 'left' ),
    'm12b_md': ( rgal['m12b_md'], 0.013, 'left' ),
    'm12c_md': ( rgal['m12c_md'], 0.01, 'left' ),
    'm11q_md': ( rgal['m11q_md'], 0.007, 'left' ),
}

In [ ]:
sim_label_locs = {
    'm12i_md': ( 17, 'left', 0 ),
    'm12b_md': ( 20, 'left', 0 ),
    'm12c_md': ( 15, 'left', 7 ),
    'm11q_md': ( 40, 'left', 5 ),
}

In [ ]:
rvir_frac = 0.1
rvir_label_locs = {
    'm12i_md': ( rvir_frac * rvir['m12i_md'], 0.042, 'left' ),
    'm12b_md': ( rvir_frac * rvir['m12b_md'], 0.039, 'left' ),
    'm12c_md': ( rvir_frac * rvir['m12c_md'], 0.036, 'left' ),
    'm11q_md': ( rvir_frac * rvir['m11q_md'], 0.033, 'left' ),
}

In [ ]:
scale = 1.
fig = plt.figure( figsize=(16,5), facecolor='w' )
ax = plt.gca()

for i, sim in enumerate( sims_to_plot_flat ):
    
    sim_color = qual_cmap[3-i]
    
    sim_label = (
        sim.split( '_' )[0] + ', ' +
#         r'$M_{\rm vir}=' + py2tex.to_tex_scientific_notation( mvir[sim], 2 ) +  r'M_\odot$, ' + 
        r'$f_{\rm thin\,disk,\,recent}$ = ' + '{:.2g}'.format( np.round( data['thin_disk_fraction_recent'][sim], 2 ) )
    )
    ax.plot(
        data['R1e5K']['points'][sim],
        data['R1e5K']['pdf'][sim],
        linewidth = 5,
        c = sim_color,
#         c = cmap( norm( data['thin_disk_fraction'][sim] ) ),
        label = sim_label,
        zorder = np.log10( mvir[sim] ),
        linestyle = linestyles[sim[:3]]
    )
    
#     # Annotate the simulation lines
#     if sim in sim_label_locs:
#         x = sim_label_locs[sim][0]
#         y = scipy.interpolate.interp1d( data['R1e5K']['points'][sim], data['R1e5K']['pdf'][sim], )( x )
#         ax.annotate(
#             text = sim.split( '_' )[0],
#             xy = ( x, y ),
#             xytext = ( 10 * ( 1 - 2 * ( sim_label_locs[sim][1] == 'right' ) ), sim_label_locs[sim][2] ),
#             xycoords = 'data',
#             textcoords = 'offset points',
#             fontsize = 20,
#             color = qual_cmap[i],
#             ha = sim_label_locs[sim][1],
#         )
    
    # Lines for the galaxy radius
    ax.axvline(
        rgal[sim],
        linewidth = 3,
        c = sim_color,
#         c = cmap( norm( data['thin_disk_fraction'][sim] ) ),
        linestyle = ':',
        zorder = np.log10( mvir[sim] ),
    )

    # Annotate the radius lines
    rgal_label = r'$R_{\rm gal}$'
    if sim in label_locs:
        text = ax.annotate(
            text = rgal_label,
            xy = label_locs[sim][:2],
            xytext = ( 5 * (1 - 2 * ( label_locs[sim][2] == 'right' ) ), -5 ),
            xycoords = 'data',
            textcoords = 'offset points',
            fontsize = 20,
            color = sim_color,
            zorder = 100,
        )
        text.set_path_effects([ patheffects.Stroke(linewidth=5, foreground='white'), patheffects.Normal() ])
        
    # Lines for the virial radius
    ax.axvline(
        rvir_frac * rvir[sim],
        linewidth = 3,
        c = sim_color,
#         c = cmap( norm( data['thin_disk_fraction'][sim] ) ),
        linestyle = '-.',
        zorder = np.log10( mvir[sim] ),
    )

    # Annotate the radius lines
    rvir_label = '{:.3g}'.format( rvir_frac ) + r' $R_{\rm vir}$'
    if sim in rvir_label_locs:
        text = ax.annotate(
            text = rvir_label,
            xy = rvir_label_locs[sim][:2],
            xytext = ( 7 * (1 - 2 * ( rvir_label_locs[sim][2] == 'right' ) ), -5 ),
            xycoords = 'data',
            textcoords = 'offset points',
            fontsize = 20,
            color = sim_color,
            ha = rvir_label_locs[sim][2],
            zorder = 100,
        )
        text.set_path_effects([ patheffects.Stroke(linewidth=5, foreground='white'), patheffects.Normal() ])

# First legend 
l = ax.legend(
    prop = { 'size': 18 },
)

# # Overwrite with second legend
# legend_labels = [ rgal_label, rvir_label, ]
# legend_linestyles = [ '-.', ':' ]
# legend_elements = [
# Line2D([0], [0], marker=None, color='k', label=legend_labels[_], markerfacecolor='k', markersize=15, linestyle=legend_linestyles[_])
# for _ in [ 0, 1, ]
# ]
# ax.legend(
# handles=legend_elements,
# prop = {'size': 18 },
# )
    
ax.set_xlim( 0, 80 )
ax.set_ylim( 0, 0.045 )

ax.set_xlabel( r'radius at which gas cools, $R_{T=10^5 {\rm K}}$ [kpc]', fontsize=22 )
ax.set_ylabel( r'PDF( $R_{T=10^5 {\rm K}}$ )', fontsize=22 )

plotting.save_fig(
    out_dir = pm['figure_dir'],
    save_file = 'R1e5K.pdf',
    fig = fig,
)

### Scaled by Rgal

In [ ]:
scale = 1.
fig = plt.figure( figsize=(16,5), facecolor='w' )
ax = plt.gca()

for i, sim in enumerate( sims_to_plot_flat ):
    ax.plot(
        data['R1e5K']['points'][sim] / rgal[sim],
        data['R1e5K']['pdf'][sim],
        linewidth = 5,
        c = qual_cmap[i],
#         c = cmap( norm( data['thin_disk_fraction'][sim] ) ),
        label = sim.split( '_' )[0],
        zorder = np.log10( mvir[sim] ),
        linestyle = linestyles[sim[:3]]
    )
    
    # Lines for the galaxy radius
    ax.axvline(
        1.,
        linewidth = 2,
        c = qual_cmap[i],
#         c = cmap( norm( data['thin_disk_fraction'][sim] ) ),
        linestyle = linestyles[sim[:3]],
        zorder = np.log10( mvir[sim] ),
    )

#     # Annotate the lines
#     if sim in label_locs:
#         ax.annotate(
#             text = r'$R_{\rm gal}$',
#             xy = label_locs[sim],
#             xytext = ( 5, -5 ),
#             xycoords = 'data',
#             textcoords = 'offset points',
#             fontsize = 20,
#             color = qual_cmap[i],
#         )

    
ax.legend(
    prop = { 'size': 18 },
)
    
ax.set_xlim( 0, 5 )
# ax.set_ylim( 0, 0.045 )

ax.set_xlabel( r'$R_{T=10^5 {\rm K}}$ / $R_{\rm gal}$', fontsize=22 )
ax.set_ylabel( r'PDF( $R_{T=10^5 {\rm K}}$ )', fontsize=22 )

# plotting.save_fig(
#     out_dir = pm['figure_dir'],
#     save_file = 'R1e5K.pdf',
#     fig = fig,
# )

## Multi Panel

In [ ]:
scale = 1.2

n_rows = len( sims_to_plot )
n_cols = len( sims_to_plot[0] )

fig = plt.figure( figsize=(6. * n_cols * scale, 3. * n_rows * scale), facecolor='w', )
fig.subplots_adjust( hspace=0.001, wspace=0.001 )
# main_ax = plt.gca()
ax_dict = fig.subplot_mosaic( sims_to_plot )

for k, row in enumerate( sims_to_plot ):
    
    for j, sim in enumerate( row ):

        ax = ax_dict[sim]
        
        r_pdf = data['R1e5K']['pdf'][sim]
        r_points = data['R1e5K']['points'][sim]
        
#         ax.plot(
#             data['R1e5K']['points'][sim],
#             data['R1e5K']['pdf'][sim],
#             linewidth = 5,
#             c = qual_cmap[i],
#     #         c = cmap( norm( data['thin_disk_fraction'][sim] ) ),
#             label = sim,
#             zorder = np.log10( mvir[sim] ),
#             linestyle = linestyles[sim[:3]]
#         )

        ax.fill_between(
            r_points,
            r_pdf,
            color = '0.5',
            linewidth = 5,
        )

        ax.set_xlim( 0, 105 ),
        ax.set_ylim( 0, 0.045 )

# Angular Distributions

## For Stars

### Recent Thin-Disk Stars

In [ ]:
fig = plt.figure( figsize=(12,6), facecolor='w' )
ax = plt.gca()

max_frac = data['thin_disk_fraction_recent'].keymax()[1]
for key, cosphi_stars in data['cosphi_stars']['thin_disk_recent'].items():
    
    ax.plot(
        data['cosphi_stars']['centers'],
        cosphi_stars,
        alpha = data['thin_disk_fraction_recent'][key] / max_frac,
        color = 'k',
        linewidth = 3,
    )
    
    # Plot particular simulations
    target_sim = 'm12m'
    if key == target_sim:
        ax.plot(
            data['cosphi_stars']['centers'],
            cosphi_stars,
            color = qual_cmap[1],
            linewidth = 5,
            label = target_sim,
            zorder = 100,
        )
    
# Calculate average pdf
avg_cosphi_stars = (
    ( data['cosphi_stars']['thin_disk_recent'] * data['thin_disk_fraction_recent'] ).array().sum( axis=0 ) / 
    data['thin_disk_fraction_recent'].array().sum()
)
# Enforce normalization
avg_cosphi_stars /= avg_cosphi_stars.sum() * ( data['cosphi_stars']['centers'][1] - data['cosphi_stars']['centers'][0] ) 
ax.plot(
    data['cosphi_stars']['centers'],
    avg_cosphi_stars,
    color = qual_cmap[0],
    linewidth = 5,
)
    
ax.set_xlim( -1, 1 )
ax.set_ylim( 0, 10 )

### All Thin-Disk Stars

In [ ]:
fig = plt.figure( figsize=(12,6), facecolor='w' )
ax = plt.gca()

max_frac = data['thin_disk_fraction'].keymax()[1]
for key, cosphi_stars in data['cosphi_stars']['thin_disk'].items():
    
    ax.plot(
        data['cosphi_stars']['centers'],
        cosphi_stars,
        alpha = data['thin_disk_fraction'][key] / max_frac,
        color = 'k',
        linewidth = 3,
    )
    
avg_cosphi_stars_all = (
    ( data['cosphi_stars']['thin_disk'] * data['thin_disk_fraction'] ).array().sum( axis=0 ) / 
    data['thin_disk_fraction'].array().sum()
)
# Enforce normalization
avg_cosphi_stars_all /= avg_cosphi_stars.sum() * ( data['cosphi_stars']['centers'][1] - data['cosphi_stars']['centers'][0] ) 
ax.plot(
    data['cosphi_stars']['centers'],
    avg_cosphi_stars_all,
    color = qual_cmap[0],
    linewidth = 5,
)
    
ax.set_xlim( -1, 1 )
ax.set_ylim( 0, 5 )

## PDF

In [ ]:
z_max = 0.15
z_min = -0.15
t_t1e5_plotted = np.array([ -0.2, -0.06, -0.03, 0., 0.03, 0.06, 0.2, ])

In [ ]:
scale = 1.2

n_rows = len( sims_to_plot )
n_cols = len( sims_to_plot[0] )

fig = plt.figure( figsize=(6. * n_cols * scale, 3. * n_rows * scale), facecolor='w', )
fig.subplots_adjust( hspace=0.001, wspace=0.02 )
# main_ax = plt.gca()
ax_dict = fig.subplot_mosaic( sims_to_plot )

for k, row in enumerate( sims_to_plot ):

    for j, sim in enumerate( row ):

        ax = ax_dict[sim]

        pdfs = data['cosphi']['hist'][sim]
        centers = data['cosphi']['points'][sim]
        t_t1e5_centers = data['cosphi']['t_t1e5_centers'][sim]

        for i, pdf in enumerate( pdfs ):
            
            # Skip over the unplotted
            if np.isclose( t_t1e5_centers[i], t_t1e5_plotted ).sum() == 0:
                continue

            z_width = z_max - z_min
            color_value = ( t_t1e5_centers[i] - z_min )/z_width
            color = palettable.scientific.diverging.Roma_3.mpl_colormap( color_value )

            if np.isclose( t_t1e5_centers[i], 0. ):
                t_t1e5_centers[i] = 0
    #             label = (
    #                 r'$t - t_{T=10^5 {\rm K}}$ =' +
    #                 ' {:.3g}'.format( t_tphase_centers[i]*1e3 ) +
    #                 r' Myr'
    #             )
            label = (
                '{:.3g}'.format( t_t1e5_centers[i]*1e3 ) +
                r' Myr'
            )

            line = ax.plot(
                centers,
                pdf,
                linewidth = 5,
                color = color,
                label = label,
        #             zorder = 10 - i,
            )

        ax.tick_params(
            axis = 'both',
            top = ax.is_first_row(),
            labeltop = ax.is_first_row(),
            bottom = ax.is_last_row(),
            labelbottom = ax.is_last_row(),
            left = ax.is_first_col(),
            labelleft = ax.is_first_col(),
            right = ax.is_last_col(),
            labelright = ax.is_last_col(),
        )
        if not ax.is_last_col():
            ax.get_xticklabels()[-3].set_horizontalalignment( 'right' )
        if not ax.is_first_col():
            ax.get_xticklabels()[0].set_horizontalalignment( 'left' )

        # Distribution curves
        ax.axhline(
            0.5,
            color = pm['background_linecolor'],
            linestyle = '-',
            linewidth = 2,
        )
#         ax.plot(
#             0.5 * np.sin( np.arccos( centers ) ),
#             np.arccos( centers ),
#         )
        ax.axvline(
            0.,
            color = pm['background_linecolor'],
            linestyle = '-',
            linewidth = 2,
        )
        
        # Comparison curves
        ax.axvline(
            pm['disk_costheta'],
            color = pm['lighter_background_linecolor'],
            linestyle = '--',
            linewidth = 2,
        )
        ax.axvline(
            -pm['disk_costheta'],
            color = pm['lighter_background_linecolor'],
            linestyle = '--',
            linewidth = 2,
        )
#         if ax.is_last_row() and ax.is_first_col():
#             annot_label = (
#                 r'$\mid z$ / $R \mid <$' +
#                 ' {:.3g}\ncontains'.format( pm['disk_costheta'] ) +
#                 r'$\approx60\%$' +
#                 ' of\nthin-disk stars'
#             )
#             ax.annotate(
#                 text = annot_label,
#                 xy = ( pm['disk_costheta'], 3.25 ),
#                 xycoords = 'data',
#                 xytext = ( 30, -10 ),
#                 textcoords = 'offset points',
#                 ha = 'left',
#                 va = 'top',
#                 fontsize = 18,
#                 arrowprops = {
#                     'arrowstyle': '-',
#                     'lw': 2,
#                     'color': pm['lighter_background_linecolor'],
#                 },
#                 color = pm['background_linecolor'],
#             )
            
#         # Comparison pdf
#         ax.plot(
#             data['cosphi_stars']['centers'],
#             avg_cosphi_stars,
#             color = pm['lighter_background_linecolor'],
#             linewidth = 2,
#         )
        
        # Sim name label
        sim_label = sim.split( '_' )[0]
#         sim_label += '\n' + r'$M_{\rm vir}=' + py2tex.to_tex_scientific_notation( mvir[sim], 2 ) +  r'M_\odot$'
        sim_label += '\n' + r'$f_{\rm thin\,disk,\,recent}$ = ' + '{:.2g}'.format( np.round( data['thin_disk_fraction_recent'][sim], 2 ) )
        ax.annotate(
            text = sim_label,
            xy = ( 0, 1 ),
            xycoords = 'axes fraction',
            xytext = ( 7, -7 ),
            textcoords = 'offset points',
            ha = 'left',
            va = 'top',
            fontsize = 15,
        )

        if ax.is_first_row() and ax.is_first_col():
            # line labels
            ax.annotate(
                text = 'isotropic\ndistribution',
                xy = ( 0.9, 0.5 ),
                xycoords = 'data',
                xytext = ( -10, 10 ),
                textcoords = 'offset points',
                ha = 'right',
                va = 'bottom',
                fontsize = 18,
                arrowprops = {
                    'arrowstyle': '-',
                    'lw': 2,
                    'color': pm['background_linecolor'],
                },
                color = pm['background_linecolor'],
            )
            ax.annotate(
                text = 'disk\nplane',
                xy = ( 0, 3.25 ),
                xycoords = 'data',
                xytext = ( 40, 10 ),
                textcoords = 'offset points',
                ha = 'left',
                va = 'top',
                fontsize = 18,
                arrowprops = {
                    'arrowstyle': '-',
                    'lw': 2,
                    'color':  pm['background_linecolor'],
                },
                color = pm['background_linecolor'],
            )

        if ax.is_first_row() and ax.is_last_col():
            ax.legend(
                prop={'size': 12},
                loc = 'upper right',
                framealpha = 1,
                title = r'$t - t_{T=10^5 {\rm K}}$',
                title_fontsize = 16,
            )

        ax.set_xlim( -1., 1. )
        ax.set_ylim( 0, 3.8 )
#         ax.set_ylim( 0., np.pi )

        if ax.is_last_row():
            ax.set_xlabel( r'$\cos\theta = z/r$', fontsize=22 )

        if ax.is_first_col():
            ax.set_ylabel( r'PDF$(z/r)$', fontsize=22 )
            
plotting.save_fig(
    out_dir = pm['figure_dir'],
    save_file = 'theta_vs_t.pdf',
    fig = fig,
)

## CDF

In [ ]:
z_min = -1.
z_max = -z_min
t_t1e5_plotted = data['cosphi']['t_t1e5_centers'][sim]

In [ ]:
scale = 1.2

n_rows = len( sims_to_plot )
n_cols = len( sims_to_plot[0] )

fig = plt.figure( figsize=(6. * n_cols * scale, 3. * n_rows * scale), facecolor='w', )
fig.subplots_adjust( hspace=0.001, wspace=0.001 )
# main_ax = plt.gca()
ax_dict = fig.subplot_mosaic( sims_to_plot )

for k, row in enumerate( sims_to_plot ):

    for j, sim in enumerate( row ):

        ax = ax_dict[sim]

        pdfs = data['cosphi']['hist'][sim]
        centers = data['cosphi']['points'][sim]
        t_t1e5_centers = data['cosphi']['t_t1e5_centers'][sim]

        for i, pdf in enumerate( pdfs ):
            
            # Skip over the unplotted
            if np.isclose( t_t1e5_centers[i], t_t1e5_plotted ).sum() == 0:
                continue

            z_width = z_max - z_min
            color_value = ( t_t1e5_centers[i] - z_min )/z_width
            color = palettable.scientific.diverging.Roma_3.mpl_colormap( color_value )

            if np.isclose( t_t1e5_centers[i], 0. ):
                t_t1e5_centers[i] = 0
    #             label = (
    #                 r'$t - t_{T=10^5 {\rm K}}$ =' +
    #                 ' {:.3g}'.format( t_tphase_centers[i]*1e3 ) +
    #                 r' Myr'
    #             )
            label = (
                '{:.3g}'.format( t_t1e5_centers[i]*1e3 ) +
                r' Myr'
            )
        
            cdf = np.cumsum( pdf )
            cdf /= cdf[-1]

            line = ax.plot(
                centers,
                cdf,
                linewidth = 5,
                color = color,
                label = label,
        #             zorder = 10 - i,
            )

        ax.tick_params(
            axis = 'both',
            top = ax.is_first_row(),
            labeltop = ax.is_first_row(),
            bottom = ax.is_last_row(),
            labelbottom = ax.is_last_row(),
            left = ax.is_first_col(),
            labelleft = ax.is_first_col(),
            right = ax.is_last_col(),
            labelright = ax.is_last_col(),
        )

        # Distribution curves
        ax.axhline(
            0.5,
            color = pm['background_linecolor'],
            linestyle = '-',
            linewidth = 2,
        )
#         ax.plot(
#             0.5 * np.sin( np.arccos( centers ) ),
#             np.arccos( centers ),
#         )
        ax.axvline(
            0.,
            color = pm['background_linecolor'],
            linestyle = '-',
            linewidth = 2,
        )
        
        # Comparison curves
        ax.axvline(
            pm['disk_costheta'],
            color = pm['lighter_background_linecolor'],
            linestyle = '--',
            linewidth = 2,
        )
        ax.axvline(
            -pm['disk_costheta'],
            color = pm['lighter_background_linecolor'],
            linestyle = '--',
            linewidth = 2,
        )
#         if ax.is_last_row() and ax.is_first_col():
#             ax.annotate(
#                 text = 'range contains\n' r'$\approx60\%$' + ' of\nthin-disk stars',
#                 xy = ( pm['disk_costheta'], 3.25 ),
#                 xycoords = 'data',
#                 xytext = ( 30, -10 ),
#                 textcoords = 'offset points',
#                 ha = 'left',
#                 va = 'top',
#                 fontsize = 18,
#                 arrowprops = {
#                     'arrowstyle': '-',
#                     'lw': 2,
#                     'color': pm['lighter_background_linecolor'],
#                 },
#                 color = pm['background_linecolor'],
#             )
            
#         # Comparison pdf
#         ax.plot(
#             data['cosphi_stars']['centers'],
#             avg_cosphi_stars,
#             color = pm['lighter_background_linecolor'],
#             linewidth = 2,
#         )
        
        # Sim name label
        sim_label = sim.split( '_' )[0]
#         sim_label += '\n' + r'$M_{\rm vir}=' + py2tex.to_tex_scientific_notation( mvir[sim], 2 ) +  r'M_\odot$'
        sim_label += '\n' + r'$f_{\rm thin\,disk,\,recent}$ = ' + '{:.2g}'.format( np.round( data['thin_disk_fraction_recent'][sim], 2 ) )
        ax.annotate(
            text = sim_label,
            xy = ( 0, 1 ),
            xycoords = 'axes fraction',
            xytext = ( 7, -7 ),
            textcoords = 'offset points',
            ha = 'left',
            va = 'top',
            fontsize = 15,
        )

        if ax.is_first_row() and ax.is_first_col():
            # line labels
            ax.annotate(
                text = 'isotropic\ndistribution',
                xy = ( 0.9, 0.5 ),
                xycoords = 'data',
                xytext = ( -10, 10 ),
                textcoords = 'offset points',
                ha = 'right',
                va = 'bottom',
                fontsize = 18,
                arrowprops = {
                    'arrowstyle': '-',
                    'lw': 2,
                    'color': pm['background_linecolor'],
                },
                color = pm['background_linecolor'],
            )
            ax.annotate(
                text = 'disk\nplane',
                xy = ( 0, 3.25 ),
                xycoords = 'data',
                xytext = ( 40, 10 ),
                textcoords = 'offset points',
                ha = 'left',
                va = 'top',
                fontsize = 18,
                arrowprops = {
                    'arrowstyle': '-',
                    'lw': 2,
                    'color':  pm['background_linecolor'],
                },
                color = pm['background_linecolor'],
            )

#         if ax.is_first_row() and ax.is_last_col():
#             ax.legend(
#                 prop={'size': 12},
#                 loc = 'upper right',
#                 framealpha = 1,
#                 title = r'$t - t_{T=10^5 {\rm K}}$',
#                 title_fontsize = 16,
#             )

        ax.set_xlim( -1., 1. )
        ax.set_ylim( 0, 1. )
#         ax.set_ylim( 0., np.pi )

        if ax.is_last_row():
            ax.set_xlabel( r'$\cos\theta = z/R$', fontsize=22 )

        if ax.is_first_col():
            ax.set_ylabel( r'CDF$(z/R)$', fontsize=22 )
            
# plotting.save_fig(
#     out_dir = pm['figure_dir'],
#     save_file = 'theta_vs_t.pdf',
#     fig = fig,
# )

# Angular Momentum Angular Distributions

## For Stars

### Recent Thin-Disk Stars

In [ ]:
fig = plt.figure( figsize=(12,6), facecolor='w' )
ax = plt.gca()

max_frac = data['thin_disk_fraction_recent'].keymax()[1]
for key, cosphi_stars in data['jz_jmag_stars']['thin_disk_recent'].items():
    
    # Enforce normalization
    cosphi_stars /= cosphi_stars.sum() * ( data['jz_jmag_stars']['centers'][key][1] - data['jz_jmag_stars']['centers'][key][0] )
    
    # Turn into a CDF
    cosphi_stars = np.cumsum( cosphi_stars )
    cosphi_stars /= cosphi_stars[-1]
    
    bins = data['jz_jmag_stars']['bins'][key]
    centers = bins[:-1] + 0.5 * ( bins[1] - bins[0] )
    ax.plot(
        centers,
        cosphi_stars,
        alpha = data['thin_disk_fraction_recent'][key] / max_frac,
        color = 'k',
        linewidth = 3,
    )

    # Plot particular simulations
    target_sim = 'm12m'
    if key == target_sim:
        ax.plot(
            centers,
            cosphi_stars,
            color = qual_cmap[1],
            linewidth = 5,
            label = target_sim,
            zorder = 100,
        )
    
# Calculate average pdf
avg_cosphi_stars = (
    ( data['jz_jmag_stars']['thin_disk_recent'] * data['thin_disk_fraction_recent'] ).array().sum( axis=0 ) / 
    data['thin_disk_fraction_recent'].array().sum()
)
# Enforce normalization
avg_cosphi_stars /= avg_cosphi_stars.sum() * ( data['jz_jmag_stars']['centers'][key][1] - data['jz_jmag_stars']['centers'][key][0] ) 

# Turn into a CDF
avg_cosphi_stars = np.cumsum( avg_cosphi_stars )
avg_cosphi_stars /= avg_cosphi_stars[-1]
avg_cosphi_stars_recent = copy.copy( avg_cosphi_stars )

ax.plot(
    centers,
    avg_cosphi_stars,
    color = qual_cmap[0],
    linewidth = 5,
    label = 'weighted average',
)

ax.legend( prop={'size': 16} )
    
ax.set_xlim( -1, 1 )
# ax.set_xlim( 0.75, 1 )
ax.set_ylim( 0, 1 )

In [ ]:
# Find minimum value of jz/jmag containing X% of stars
X = 90.
cdf = avg_cosphi_stars
x_percentile = scipy.interpolate.interp1d( 1. - cdf, centers )( X / 100. )
print( '{:.3g}'.format( x_percentile ) )

### All Thin-Disk Stars

In [ ]:
fig = plt.figure( figsize=(12,6), facecolor='w' )
ax = plt.gca()

max_frac = data['thin_disk_fraction'].keymax()[1]
for key, cosphi_stars in data['jz_jmag_stars']['thin_disk'].items():
    
    # Enforce normalization
    cosphi_stars /= cosphi_stars.sum() * ( data['jz_jmag_stars']['centers'][key][1] - data['jz_jmag_stars']['centers'][key][0] )
    
    # Turn into a CDF
    cosphi_stars = np.cumsum( cosphi_stars )
    cosphi_stars /= cosphi_stars[-1]
    
    bins = data['jz_jmag_stars']['bins'][key]
    centers = bins[:-1] + 0.5 * ( bins[1] - bins[0] )
    ax.plot(
        centers,
        cosphi_stars,
        alpha = data['thin_disk_fraction'][key] / max_frac,
        color = 'k',
        linewidth = 3,
    )

    # Plot particular simulations
    target_sim = 'm12m'
    if key == target_sim:
        ax.plot(
            centers,
            cosphi_stars,
            color = qual_cmap[1],
            linewidth = 5,
            label = target_sim,
            zorder = 100,
        )
    
# Calculate average pdf
avg_cosphi_stars = (
    ( data['jz_jmag_stars']['thin_disk'] * data['thin_disk_fraction'] ).array().sum( axis=0 ) / 
    data['thin_disk_fraction_recent'].array().sum()
)
# Enforce normalization
avg_cosphi_stars /= avg_cosphi_stars.sum() * ( data['jz_jmag_stars']['centers'][key][1] - data['jz_jmag_stars']['centers'][key][0] ) 

# Turn into a CDF
avg_cosphi_stars = np.cumsum( avg_cosphi_stars )
avg_cosphi_stars /= avg_cosphi_stars[-1]

ax.plot(
    centers,
    avg_cosphi_stars,
    color = qual_cmap[0],
    linewidth = 5,
    label = 'weighted average',
)

ax.legend( prop={'size': 16} )
    
ax.set_xlim( -1, 1 )
ax.set_ylim( 0, 1 )

## Horizontal

In [ ]:
# t_t1e5_plotted = np.array([ -1.005, 0.0, 0.150 ])
t_t1e5_plotted = np.arange( -1., 0.5 + 0.01, 0.025 )
# t_t1e5_plotted = np.arange( -1.005, 0.00, 0.090 )
t_t1e5_emphasized = [ -1.0, -0.5, -0.2, 0., 0.2, 0.5 ]
z_min = -1.
z_max = - z_min
is_cdf = True

### CDF

In [ ]:
scale = 1.2

n_rows = len( sims_to_plot )
n_cols = len( sims_to_plot[0] )

fig = plt.figure( figsize=(6. * n_cols * scale, 3. * n_rows * scale), facecolor='w', )
fig.subplots_adjust( hspace=0.001, wspace=0.02 )
ax_dict = fig.subplot_mosaic( sims_to_plot )

for k, row in enumerate( sims_to_plot ):

    for j, sim in enumerate( row ):

        ax = ax_dict[sim]

        pdfs = data['jzjmag']['pdf'][sim]
        centers = data['jzjmag']['points'][sim]
        t_t1e5_centers = data['jzjmag']['t_t1e5_centers'][sim]

        outlined_cdfs = []
        for i, pdf in enumerate( pdfs ):
            
            if np.isclose( t_t1e5_centers[i], t_t1e5_plotted ).sum() == 0:
                continue

            z_width = z_max - z_min
            color_value = ( t_t1e5_centers[i] - z_min )/z_width
            color = palettable.scientific.diverging.Roma_9.mpl_colormap( color_value )

            if np.isclose( t_t1e5_centers[i], 0. ).sum() != 0:
                t_t1e5_centers[i] = 0.

            if np.isclose( t_t1e5_centers[i], t_t1e5_emphasized ).sum() != 0:
                zorder = 1000 - i
                linestyle = '-'
                linewidth = 5
                    
                # Label is straightforward, but needs to handle edge cases
                if np.isclose( t_t1e5_centers[i], -1. ).sum() == 0:
                    label = (
                        '{:.3g}'.format( t_t1e5_centers[i]*1e3 ) +
                        r' Myr'
                    )
                else:
                    label = '-1 Gyr'
                
                # Outline everything except for the two reference points
                if np.isclose( t_t1e5_centers[i], [ -1., -0.5, 0.5 ] ).sum() == 0:
                    outline = True
                    outline_linewidth = linewidth + 5
                else:
                    outline = False
                    zorder = -i
            else:
                label = None
                linewidth = 2
                outline = False
                zorder = -i
            
            # Turn into a cdf
            if is_cdf:
                ys = np.cumsum( pdf )
                ys /= ys[-1]
            else:
                ys = pdf
            
            line = ax.plot(
                centers,
                ys,
                linewidth = linewidth,
                color = color,
                linestyle = linestyle,
                zorder = zorder,
                label = label,
            )[0]
            if outline:
                line.set_path_effects([patheffects.Stroke(linewidth=outline_linewidth, foreground='black', ),
                       patheffects.Normal()])
                outlined_cdfs.append( ys )

        # Avoid overlapping ticks
        ax.tick_params(
            axis = 'both',
            top = ax.is_first_row(),
            labeltop = ax.is_first_row(),
            bottom = ax.is_last_row(),
            labelbottom = ax.is_last_row(),
            left = ax.is_first_col(),
            labelleft = ax.is_first_col(),
            right = ax.is_last_col(),
            labelright = ax.is_last_col(),
        )
        if not ax.is_last_col():
            ax.get_xticklabels()[-3].set_horizontalalignment( 'right' )
        if not ax.is_first_col():
            ax.get_xticklabels()[0].set_horizontalalignment( 'left' )
        if not ax.is_last_row():
            ax.get_yticklabels()[0].set_verticalalignment( 'bottom' )
        if not ax.is_first_row():
            ax.get_yticklabels()[-1].set_verticalalignment( 'top' )

        # Distribution curves
        ax.plot(
            [ -1, 1 ],
            [ 0, 1 ],
            color = pm['background_linecolor'],
            linewidth = 2,
        )
        if ax.is_first_row() and ax.is_first_col():
            ax.annotate(
                text = 'isotropic\ndistribution',
                xy = ( -0.5, 0.25 ),
                xycoords = 'data',
                xytext = ( -10, 10 ),
                textcoords = 'offset points',
                ha = 'right',
                va = 'bottom',
                fontsize = 18,
                arrowprops = {
                    'arrowstyle': '-',
                    'lw': 2,
                    'color': pm['background_linecolor'],
                },
                color = pm['background_linecolor'],
            )
        
        # Comparison curves
        ax.axvline(
            pm['disk_jzjmag'],
            color = pm['lighter_background_linecolor'],
            linestyle = '--',
            linewidth = 2,
        )
        if ax.is_last_row() and ax.is_last_col():
            annot_label = (
                r'$j_z$ / $\mid \vec j \mid >$' +
                ' {:.3g}\ncontains'.format( pm['disk_jzjmag'] ) +
                r'$\approx90\%$' +
                ' of\nthin-disk stars'
            )
            text = ax.annotate(
                text = annot_label,
                xy = ( pm['disk_jzjmag'], 0.1 ),
                xycoords = 'data',
                xytext = ( -30, 10 ),
                textcoords = 'offset points',
                ha = 'right',
                va = 'bottom',
                fontsize = 18,
                arrowprops = {
                    'arrowstyle': '-',
                    'lw': 2,
                    'color': pm['lighter_background_linecolor'],
                },
                color = pm['background_linecolor'],
            )
            text.set_path_effects([patheffects.Stroke(linewidth=outline_linewidth, foreground='w', ),
                       patheffects.Normal()])
            
#         # Annotation of change between outlined cdfs
#         if ax.is_first_row() and ax.is_first_col():
#             x = 0.5
#             y0 = scipy.interpolate.interp1d( centers, outlined_cdfs[0] )( x )
#             y1 = scipy.interpolate.interp1d( centers, outlined_cdfs[1] )( x )
#             dy = ( y1 - y0 )*0.9
#             ax.arrow(
#                 x,
#                 y0,
#                 0,
#                 dy,
#                 zorder = 300,
#                 width = 0.015,
#                 color = pm['background_linecolor'],
#                 length_includes_head = True,
#             )
#             text = ax.annotate(
#                 text = r'200 Myrs',
#                 xy = ( x, y0 + dy * 0.25 ),
#                 xycoords = 'data',
#                 xytext = ( -10, -30 ),
#                 textcoords = 'offset points',
#                 ha = 'right',
#                 va = 'center',
#                 fontsize = 14,
#                 arrowprops = {
#                     'arrowstyle': '-',
#                     'lw': 2,
#                     'color': pm['background_linecolor'],
#                 },
#                 color = pm['background_linecolor'],
#                 zorder = 3000,
#             )
# #             text.set_path_effects([patheffects.Stroke(linewidth=1, foreground='w', ),
# #                        patheffects.Normal()])
            
#         # Comparison pdf
#         bins = data['jz_jmag_stars']['bins'][sim]
#         centers = bins[:-1] + 0.5 * ( bins[1] - bins[0] )
#         ax.plot(
#             centers,
#             avg_cosphi_stars_recent,
#             color = pm['lighter_background_linecolor'],
#             linewidth = 2,
#         )
        
        # Sim name label
        sim_label = sim.split( '_' )[0]
#         sim_label += '\n' + r'$M_{\rm vir}=' + py2tex.to_tex_scientific_notation( mvir[sim], 2 ) +  r'M_\odot$'
        sim_label += '\n' + r'$f_{\rm thin\,disk,\,recent}$ = ' + '{:.2g}'.format( np.round( data['thin_disk_fraction_recent'][sim], 2 ) )
        ax.annotate(
            text = sim_label,
            xy = ( 0, 1 ),
            xycoords = 'axes fraction',
            xytext = ( 7, -7 ),
            textcoords = 'offset points',
            ha = 'left',
            va = 'top',
            fontsize = 15,
        )

        if ax.is_first_row() and ax.is_last_col():
            l = ax.legend(
                prop={'size': 12},
                loc = 'lower left',
                framealpha = 1,
                title = r'$t - t_{T=10^5 {\rm K}}$',
                title_fontsize = 16,
            )
            l.set_zorder( 2000 )

        ax.set_xlim( -1., 1. )
        if is_cdf:
            ax.set_ylim( 0, 1. )
        else:
            ax.set_ylim( 0, 5 )
#         ax.set_ylim( 0., np.pi )

        if ax.is_last_row():
            ax.set_xlabel( r'$\cos\theta_j$ = $j_z$ / $\mid \vec j \mid$', fontsize=22 )

        if ax.is_first_col():
            ax.set_ylabel( r'CDF( $j_z$ / $\mid \vec j \mid$ )', fontsize=22 )
            
# plotting.save_fig(
#     out_dir = pm['figure_dir'],
#     save_file = 'jzjmag_vs_t.pdf',
#     fig = fig,
# )

### Fraction Above

In [ ]:
scale = 1.2

n_rows = len( sims_to_plot )
n_cols = len( sims_to_plot[0] )

fig = plt.figure( figsize=(6. * n_cols * scale, 3. * n_rows * scale), facecolor='w', )
fig.subplots_adjust( hspace=0.045, wspace=0.02 )
ax_dict = fig.subplot_mosaic( sims_to_plot )

for k, row in enumerate( sims_to_plot ):

    for j, sim in enumerate( row ):

        ax = ax_dict[sim]

        pdfs = data['jzjmag']['pdf'][sim]
        centers = data['jzjmag']['points'][sim]
        t_t1e5_centers = data['jzjmag']['t_t1e5_centers'][sim]

        outlined_cdfs = []
        for i, pdf in enumerate( pdfs ):
            
            if np.isclose( t_t1e5_centers[i], t_t1e5_plotted ).sum() == 0:
                continue

            z_width = z_max - z_min
            color_value = ( t_t1e5_centers[i] - z_min )/z_width
            color = palettable.scientific.diverging.Roma_9.mpl_colormap( color_value )

            if np.isclose( t_t1e5_centers[i], 0. ).sum() != 0:
                t_t1e5_centers[i] = 0.

            if np.isclose( t_t1e5_centers[i], t_t1e5_emphasized ).sum() != 0:
                zorder = 1000 - i
                linestyle = '-'
                linewidth = 5
                    
                # Label is straightforward, but needs to handle edge cases
                if np.isclose( t_t1e5_centers[i], -1. ).sum() == 0:
                    label = (
                        '{:.3g}'.format( t_t1e5_centers[i]*1e3 ) +
                        r' Myr'
                    )
                else:
                    label = '-1 Gyr'
                
                # Outline everything except for the two reference points
                if np.isclose( t_t1e5_centers[i], [ -1., -0.5, 0.5 ] ).sum() == 0:
                    outline = True
                    outline_linewidth = linewidth + 3
                else:
                    outline = False
                    zorder = -i
            else:
                label = None
                linewidth = 2
                outline = False
                zorder = -i
            
            # Turn into mass fraction
            ys = np.cumsum( pdf )
            ys /= ys[-1]
            ys = 1. - ys
                
            line = ax.plot(
                centers,
                ys,
                linewidth = linewidth,
                color = color,
                linestyle = linestyle,
                zorder = zorder,
                label = label,
            )[0]
            if outline:
                line.set_path_effects([patheffects.Stroke(linewidth=outline_linewidth, foreground='black', ),
                       patheffects.Normal()])
                outlined_cdfs.append( ys )

        # Avoid overlapping ticks
        ax.tick_params(
            axis = 'both',
            top = ax.is_first_row(),
            labeltop = ax.is_first_row(),
            bottom = ax.is_last_row(),
            labelbottom = ax.is_last_row(),
            left = ax.is_first_col(),
            labelleft = ax.is_first_col(),
            right = ax.is_last_col(),
            labelright = ax.is_last_col(),
        )
        if not ax.is_last_col():
            ax.get_xticklabels()[-3].set_horizontalalignment( 'right' )
        if not ax.is_first_col():
            ax.get_xticklabels()[0].set_horizontalalignment( 'left' )
        if not ax.is_last_row():
            ax.get_yticklabels()[0].set_verticalalignment( 'bottom' )
        if not ax.is_first_row():
            ax.get_yticklabels()[-1].set_verticalalignment( 'top' )

        # Distribution curves
        ax.plot(
            [ -1, 1 ],
            [ 1, 0 ],
            color = pm['background_linecolor'],
            linewidth = 2,
        )
        if ax.is_first_row() and ax.is_first_col():
            ax.annotate(
                text = 'isotropic\ndistribution',
                xy = ( -0.5, 0.75 ),
                xycoords = 'data',
                xytext = ( -10, -10 ),
                textcoords = 'offset points',
                ha = 'right',
                va = 'top',
                fontsize = 18,
                arrowprops = {
                    'arrowstyle': '-',
                    'lw': 2,
                    'color': pm['background_linecolor'],
                },
                color = pm['background_linecolor'],
            )
        
        # Comparison curves
        ax.axvline(
            pm['disk_jzjmag'],
            color = pm['lighter_background_linecolor'],
            linestyle = '--',
            linewidth = 2,
        )
        if ax.is_last_row() and ax.is_last_col():
            annot_label = (
                r'$j_z$ / $\mid \vec j \mid >$' +
                ' {:.3g}\ncontains'.format( pm['disk_jzjmag'] ) +
                r'$\approx90\%$' +
                ' of\nthin-disk stars'
            )
            text = ax.annotate(
                text = annot_label,
                xy = ( pm['disk_jzjmag'], 0.5 ),
                xycoords = 'data',
                xytext = ( -5, 30 ),
                textcoords = 'offset points',
                ha = 'right',
                va = 'bottom',
                fontsize = 18,
                arrowprops = {
                    'arrowstyle': '-',
                    'lw': 2,
                    'color': pm['lighter_background_linecolor'],
                },
                color = pm['background_linecolor'],
            )
#             text.set_path_effects([patheffects.Stroke(linewidth=10, foreground='w', ),
#             patheffects.Normal()])
            
        # Disky and co-rotating lines
        jzjmag_vals = [ 0., 0.5 ]
        for i, annot_label in enumerate(['co-rotating', ]):
            ax.axvline(
                jzjmag_vals[i],
                color = pm['lighter_background_linecolor'],
                linestyle = '--',
                linewidth = 2,
                zorder = -10,
            )
            if ax.is_last_row() and ax.is_first_col():
                arrow = ax.annotate(
                    text = '',
                    xy = ( jzjmag_vals[i] + 0.175, 0.15 ),
                    xytext = ( -35, 0 ),
                    xycoords = 'data',
                    textcoords = 'offset points',
                    zorder = 300,
                    color = pm['background_linecolor'],
                    ha = 'right',
                    va = 'center',
                    arrowprops = dict( arrowstyle = 'wedge', color = pm['background_linecolor'], ),
                )
                text.set_path_effects([patheffects.Stroke(linewidth=5, foreground='w', ),
                patheffects.Normal()])
                text = ax.annotate(
                    text = annot_label,
                    xy = ( jzjmag_vals[i], 0. ),
                    xycoords = 'data',
                    xytext = ( 5, 5 ),
                    textcoords = 'offset points',
                    ha = 'left',
                    va = 'bottom',
                    fontsize = 18,
    #                 arrowprops = {
    #                     'arrowstyle': '-',
    #                     'lw': 2,
    #                     'color': pm['lighter_background_linecolor'],
    #                 },
                    color = pm['background_linecolor'],
                )
                text.set_path_effects([patheffects.Stroke(linewidth=10, foreground='w', ),
                patheffects.Normal()])
        
#         # Annotation of change between outlined cdfs
#         if ax.is_first_row() and ax.is_first_col():
#             x = 0.5
#             y0 = scipy.interpolate.interp1d( centers, outlined_cdfs[0] )( x )
#             y1 = scipy.interpolate.interp1d( centers, outlined_cdfs[1] )( x )
#             dy = ( y1 - y0 )*0.9
#             ax.arrow(
#                 x,
#                 y0,
#                 0,
#                 dy,
#                 zorder = 300,
#                 width = 0.015,
#                 color = pm['background_linecolor'],
#                 length_includes_head = True,
#             )
#             text = ax.annotate(
#                 text = r'200 Myrs',
#                 xy = ( x, y0 + dy * 0.25 ),
#                 xycoords = 'data',
#                 xytext = ( -10, 30 ),
#                 textcoords = 'offset points',
#                 ha = 'right',
#                 va = 'center',
#                 fontsize = 14,
#                 arrowprops = {
#                     'arrowstyle': '-',
#                     'lw': 2,
#                     'color': pm['background_linecolor'],
#                 },
#                 color = pm['background_linecolor'],
#                 zorder = 3000,
#             )
# #             text.set_path_effects([patheffects.Stroke(linewidth=1, foreground='w', ),
# #                        patheffects.Normal()])
            
#         # Comparison pdf
#         bins = data['jz_jmag_stars']['bins'][sim]
#         centers = bins[:-1] + 0.5 * ( bins[1] - bins[0] )
#         ax.plot(
#             centers,
#             avg_cosphi_stars_recent,
#             color = pm['lighter_background_linecolor'],
#             linewidth = 2,
#         )
        
        # Sim name label
        sim_label = sim.split( '_' )[0]
#         sim_label += '\n' + r'$M_{\rm vir}=' + py2tex.to_tex_scientific_notation( mvir[sim], 2 ) +  r'M_\odot$'
        sim_label += '\n' + r'$f_{\rm thin\,disk,\,recent}$ = ' + '{:.2g}'.format( np.round( data['thin_disk_fraction_recent'][sim], 2 ) )
        ax.annotate(
            text = sim_label,
            xy = ( 0, 0 ),
            xycoords = 'axes fraction',
            xytext = ( 7, 7 ),
            textcoords = 'offset points',
            ha = 'left',
            va = 'bottom',
            fontsize = 15,
        )

        if ax.is_first_row() and ax.is_last_col():
            l = ax.legend(
                prop={'size': 12},
                loc = 'upper left',
                framealpha = 1,
                title = r'$t - t_{T=10^5 {\rm K}}$',
                title_fontsize = 16,
            )
            l.set_zorder( 2000 )

        ax.set_xlim( -1., 1. )
        if is_cdf:
            ax.set_ylim( 0, 1. )
        else:
            ax.set_ylim( 0, 5 )
#         ax.set_ylim( 0., np.pi )

        if ax.is_last_row():
            ax.set_xlabel( r'$\cos\theta_j$ = $j_z$ / $\mid \vec j \mid$', fontsize=22 )

        if ax.is_first_col():
            ax.set_ylabel( r'fraction > $j_z$ / $\mid \vec j \mid$', fontsize=22, )
            
plotting.save_fig(
    out_dir = pm['figure_dir'],
    save_file = 'jzjmag_vs_t.pdf',
    fig = fig,
)